In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

import requests
import json
from random import randint
import os
import time
import json
import pickle

In [15]:
%cd ..
%cd demo Mativ
all_anime_df = pd.read_csv(os.path.join("data", "tags_anime.csv"))
%cd ..
%cd demo Victor
all_anime_df

e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Mativ
e:\Github\datos\Proyecto-Grupo-12
e:\Github\datos\Proyecto-Grupo-12\demo Victor


In [18]:
detalles_anime = {}
for i in range(1, 16):
    path = os.path.join("data", f"detalles_anime_{i}.pickle")
    with open(path, "rb") as archivo:
        detalles_anime = detalles_anime | pickle.load(archivo)

In [24]:
pd.set_option('display.max_rows', 100)

In [46]:
import re

dummy_df = all_anime_df.copy()

dummy_df["Anime"] = dummy_df["Anime"].str.lower()
dummy_df["Anime"] = dummy_df["Anime"].str.replace(r"[^a-zA-Z0-9]", "", regex=True)

pd.options.mode.chained_assignment = None  # default='warn'
titulos = pd.DataFrame()
not_found = {}
for anime_id, value in detalles_anime.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]
    for titulo in lista_titulos:
        mini = all_anime_df[dummy_df["Anime"] == re.sub("[^a-zA-Z0-9]", "", titulo.lower())]
        if not mini.empty:
            break
    else: # No se encontro el anime
        not_found[anime_id] = value
        continue
    mini["MAL_ID"] = anime_id
    titulos = pd.concat([titulos, mini])

In [39]:
titulos

,Anime,URL,MAL_ID
112,Cowboy Bebop,https://www.anime-planet.com/anime/cowboy-bebop,1
13925,Puzzle & Dragons X,https://www.anime-planet.com/anime/puzzle-and-...,32772
353,Cowboy Bebop: The Movie,https://www.anime-planet.com/anime/cowboy-bebo...,5
422,Trigun,https://www.anime-planet.com/anime/trigun,6
4663,Witch Hunter Robin,https://www.anime-planet.com/anime/witch-hunte...,7
...,...,...,...
15847,Chitose Get You!!,https://www.anime-planet.com/anime/chitose-get...,14277
13013,Vividred Operation,https://www.anime-planet.com/anime/vividred-op...,14283
1379,"Say ""I love you.""",https://www.anime-planet.com/anime/say-i-love-you,14289
6533,Psychic School Wars,https://www.anime-planet.com/anime/psychic-sch...,14293


In [ ]:
from difflib import SequenceMatcher

# dataframe de all_anime_df sin titulos
some_anime_df = all_anime_df[~all_anime_df.index.isin(titulos.index)]

dummy_df_2 = some_anime_df.copy()
dummy_df_2["Anime"] = dummy_df_2["Anime"].str.lower()
dummy_df_2["Anime"] = dummy_df_2["Anime"].str.replace(r"[^a-zA-Z0-9 ]", "", regex=True) # En diferencia con el anterior si se mantendra un espacio.

not_found_2 = {}
titulos_2 = pd.DataFrame()

for anime_id, value in not_found.items():
    details = value["detalles"]
    lista_titulos = [details["title"]] + (details["alternative_titles"]["synonyms"]) + [(details["alternative_titles"]["en"])]

    max_score = 0
    for titulo in lista_titulos:
        # Se busca el titulo mas parecido
        ratios = dummy_df_2["Anime"].apply(lambda x: SequenceMatcher(None, x, re.sub("[^a-zA-Z0-9]", "", titulo.lower())).ratio())
        score = ratios.max()
        score_id = ratios.idxmax()
        mini = some_anime_df[dummy_df_2.index == score_id]
        if not mini.empty:
            break
    else: # No se encontro el anime
        not_found[anime_id] = value
        continue
    mini["MAL_ID"] = anime_id
    titulos_2 = pd.concat([titulos_2, mini])